For this project we need:
1. access the dataset with data from 01/01/1980 - 01/01/2020
2. find the average of each value for each day

In [1]:
import datetime
import pandas as pd

In [2]:
url = 'weather_output.csv'
df = pd.read_csv(url)
df.drop(df.columns[0], axis=1, inplace=True)

First we need to be able to find all values of 1st of january

In [3]:
averaged_df = pd.DataFrame()

In [4]:
def create_day_month_columns():
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year

In [15]:
def get_average_values():
    values_dict = {}
    for month in range(1,13):
        for day in range(1,32):
            condition1 = df['day'] == day
            condition2 = df['month'] == month
            filter_df = df[condition1 & condition2]
            if filter_df.empty:
                continue
            date = f'{day}/{month}'
            t_mean = filter_df[filter_df.columns[2]].mean() 
            t_max_mean = filter_df[filter_df.columns[3]].mean() 
            t_min_mean = filter_df[filter_df.columns[4]].mean()
            precipitation_mean = filter_df[filter_df.columns[5]].mean()
            rain_mean = filter_df[filter_df.columns[6]].mean()
            snow_mean = filter_df[filter_df.columns[7]].mean()
            precipitation_hours_mean = filter_df[filter_df.columns[8]].mean()
            wind_speed_mean = filter_df[filter_df.columns[9]].mean()
            wind_gusts_mean = filter_df[filter_df.columns[10]].mean()
            shortwave_radiation_mean = filter_df[filter_df.columns[12]].mean()
            values_dict[date] = {'temperature_mean': t_mean,
                                'max_temperature_mean': t_max_mean,
                                'min_temperature_mean': t_min_mean,
                                'precipitation_mean': precipitation_mean,
                                'rain_mean': rain_mean,
                                'snow_mean': snow_mean,
                                'precipitation_hours_mean': precipitation_hours_mean,
                                'wind_speed_mean': wind_speed_mean,
                                'wind_gusts_mean': wind_gusts_mean,
                                'shortwave_radiation_mean': shortwave_radiation_mean
                                }
    return_df = pd.DataFrame.from_dict(values_dict, orient='index')
    return return_df


In [ ]:
def get_averaged_df():
    create_day_month_columns()
    averaged_df = get_average_values()
    averaged_df.to_csv('weather_averaged.csv')

In [31]:
def map_averages():
    mean_columns = ['temperature_mean',
                    'max_temperature_mean',
                    'min_temperature_mean',
                    'precipitation_mean',
                    'rain_mean',
                    'snow_mean',
                    'precipitation_hours_mean',
                    'wind_speed_mean',
                    'wind_gusts_mean',
                    'shortwave_radiation_mean'
    ]
    df['concat_dates'] = df['day'].astype(str)+'/'+df['month'].astype(str)
    df['concat_dates']

    for mean in mean_columns:
        df[mean] = df['concat_dates'].map(averaged_df[mean])

In [39]:
def calculate_variations():
    value_mean_pairs = [
        ('temperature_2m_mean', 'temperature_mean'),
        ('temperature_2m_max', 'max_temperature_mean'),
        ('temperature_2m_min', 'min_temperature_mean'),
        ('precipitation_sum', 'precipitation_mean'),
        ('rain_sum', 'rain_mean'),
        ('snowfall_sum', 'snow_mean'),
        ('precipitation_hours', 'precipitation_hours_mean'),
        ('wind_speed_10m_max', 'wind_speed_mean'),
        ('wind_gusts_10m_max', 'wind_gusts_mean'),
        ('shortwave_radiation_sum', 'shortwave_radiation_mean'),
    ]

    for actual, mean in value_mean_pairs:
        percent_var_col = f"{actual}_percent_var"
        diff_var_col = f"{actual}__var"
        
        df[percent_var_col] = (df[actual] / df[mean])*100
        df[diff_var_col] = df[actual] - df[mean]

In [ ]:
def order_columns():
    ordered_columns = [
        # Date
        'day', 'month', 'year',
        # Temperature
        'temperature_2m_mean', 'temperature_mean', 'temperature_2m_mean_percent_var', 'temperature_2m_mean__var',
        'temperature_2m_min', 'min_temperature_mean', 'temperature_2m_min_percent_var', 'temperature_2m_min__var',
        'temperature_2m_max', 'max_temperature_mean', 'temperature_2m_max_percent_var', 'temperature_2m_max__var',

        # Precipitation
        'precipitation_sum', 'precipitation_mean', 'precipitation_sum_percent_var', 'precipitation_sum__var',

        # Rain
        'rain_sum', 'rain_mean', 'rain_sum_percent_var', 'rain_sum__var',

        # Snow
        'snowfall_sum', 'snow_mean', 'snowfall_sum_percent_var', 'snowfall_sum__var',

        # Precipitation hours
        'precipitation_hours', 'precipitation_hours_mean', 'precipitation_hours_percent_var', 'precipitation_hours__var',

        # Winds
        'wind_speed_10m_max', 'wind_speed_mean', 'wind_speed_10m_max_percent_var', 'wind_speed_10m_max__var',
        'wind_gusts_10m_max', 'wind_gusts_mean', 'wind_gusts_10m_max_percent_var', 'wind_gusts_10m_max__var',

        # Shortwave radiation
        'shortwave_radiation_sum', 'shortwave_radiation_mean',
        'shortwave_radiation_sum_percent_var', 'shortwave_radiation_sum__var'
        ]

    # Reorganize the columns
    df = df[[col for col in ordered_columns if col in df.columns]]

In [41]:
def main():
    get_averaged_df()
    map_averages()
    calculate_variations()
    order_columns
    df.to_csv('full_weather.csv')

In [44]:
df

,day,month,year,temperature_2m_mean,temperature_mean,temperature_2m_mean_percent_var,temperature_2m_mean__var,temperature_2m_min,min_temperature_mean,temperature_2m_min_percent_var,...,wind_speed_10m_max_percent_var,wind_speed_10m_max__var,wind_gusts_10m_max,wind_gusts_mean,wind_gusts_10m_max_percent_var,wind_gusts_10m_max__var,shortwave_radiation_sum,shortwave_radiation_mean,shortwave_radiation_sum_percent_var,shortwave_radiation_sum__var
0,1,1,1980,-1.929167,4.995909,-0.386149,-6.925075,-4.0000,2.537220,-1.576529,...,0.692117,-7.284304,28.440000,47.221463,0.602269,-18.781463,2.58,2.080976,1.239803,0.499024
1,2,1,1980,-2.406250,4.867831,-0.494317,-7.274081,-4.5500,2.426738,-1.874945,...,0.694593,-7.466477,29.519999,49.022999,0.602166,-19.503000,3.99,2.549500,1.565013,1.440500
2,3,1,1980,2.079167,4.440748,0.468202,-2.361581,-4.5000,1.895487,-2.374059,...,0.936039,-1.630634,45.000000,50.408999,0.892698,-5.408999,0.69,2.273250,0.303530,-1.583250
3,4,1,1980,7.283333,4.638925,1.570048,2.644408,2.9500,1.892987,1.558383,...,0.959786,-1.008224,44.639996,49.787999,0.896602,-5.148003,1.75,2.322250,0.753580,-0.572250
4,5,1,1980,3.785417,5.098613,0.742441,-1.313196,1.9500,2.332988,0.835838,...,0.943146,-1.345657,38.880000,46.394999,0.838021,-7.514999,3.54,2.184750,1.620323,1.355250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14606,28,12,2019,7.802333,4.444394,1.755545,3.357940,6.3065,1.825487,3.454694,...,0.640913,-7.680443,23.759998,42.326999,0.561344,-18.567001,1.77,2.472500,0.715875,-0.702500
14607,29,12,2019,5.929417,4.315175,1.374085,1.614242,3.4065,1.835488,1.855910,...,0.751717,-5.264096,27.359999,42.795000,0.639327,-15.435001,3.47,2.247000,1.544281,1.223000
14608,30,12,2019,5.448166,5.128040,1.062427,0.320127,2.2065,2.829237,0.779892,...,0.724358,-6.307572,28.440000,44.756999,0.635431,-16.316999,3.77,2.031250,1.856000,1.738750
14609,31,12,2019,5.952333,5.185800,1.147814,0.766534,2.2065,2.874237,0.767682,...,0.754503,-5.525245,28.440000,43.325999,0.656419,-14.885999,1.49,2.077000,0.717381,-0.587000
